<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/101_TxtSummarizerAgent_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## ✅ Next Step: Implement Tools (starting with `create_plan`)



### 🛠 Tool: `create_plan`

This is our “mental warmup” tool. Its job:

* Read the goal from memory
* Create a simple plan (list of steps)
* Store that plan back in memory

---

## ✍️ Step 1: Define Tool Function

```python
def create_plan(ctx):
    goal = ctx.memory.get("goal")

    plan = [
        "Read the target file",
        "Create a prompt for summarization",
        "Use LLM to generate summary",
        "Save the summary to the output folder",
        "Log the completion"
    ]

    ctx.memory.set("plan", plan)

    return {"message": "Plan created.", "steps": plan}
```

---

## ✅ Notes

* `ctx.memory.get("goal")` → We expect the goal was injected earlier by the agent setup
* We return a confirmation + the steps (optional but helpful for debug or transparency)
* This is a **zero-dependency** tool: no extra config or files needed



## 🧠 Why Hardcoding the Plan Isn’t Ideal

* Hardcoded steps = the agent can only do one thing.
* It defeats the purpose of a reusable `create_plan` tool.
* We want agents that can adapt to **any goal**, even ones we haven’t thought of yet.

---

## ✅ The Better Design: Use the LLM to Generate the Plan

Let’s refactor:

### 🔧 Improved `create_plan(ctx)`:

```python
def create_plan(ctx):
    goal = ctx.memory.get("goal")

    prompt = f"""You are an expert task planner. Given the goal below, break it down into a clear, short list of steps.

Goal: {goal}

Respond with a numbered list of steps."""

    response = ctx.llm.complete(prompt)

    # Optional: Parse into list of steps
    steps = response.strip().split("\n")

    ctx.memory.set("plan", steps)

    return {"message": "Plan created from goal.", "steps": steps}
```

---

## ✅ Why This Is Better

| Hardcoded Plan            | LLM-Generated Plan              |
| ------------------------- | ------------------------------- |
| Fixed and rigid           | Flexible and dynamic            |
| Only works for 1 use case | Adapts to new goals             |
| Not reusable              | Can power many different agents |

---

## 💡 Bonus Insight

This pattern — where the LLM **only handles the hard thinking** — is what your teacher emphasized.

Here:

* We inject the goal
* The LLM does the reasoning (“how should I achieve that?”)
* The agent just **records** and **executes** the plan

---

## ✅ TL;DR

> Yes, `create_plan` should be an LLM tool. The goal goes in, and the agent gets back steps.
> We store those steps in memory, and then proceed.





## ✅ “Inject the goal” — What Does That Mean?

You’re right:

> It means we **set the goal before** the tool is ever called — so the tool can just read it from context.

It’s **not** hardcoded.
It’s **not** asked for at runtime.
It’s **already there** in `ctx.memory`.

---

## ✅ Is This Dependency Injection?

Yes — though it’s a special kind:

* Classic dependency injection is for **code or config** (like folders, clocks, models)
* But here, we’re injecting **initial memory state** (the `goal`) into the agent’s `ActionContext` before it starts

So while it’s not dependency injection in the strict OOP sense, it *follows the same principle*:

> 🔄 **Give each unit (tool or agent) what it needs — from the outside — instead of baking it in.**

---

## 🧳 Where Does the Goal Live?

When building the agent, you might do this:

```python
ctx = ActionContext(...)
ctx.memory.set("goal", "Summarize the content of a text file.")
```

Now the tool can access that goal like this:

```python
goal = ctx.memory.get("goal")
```

It’s been **injected into the memory layer** — just like folders are injected into `config`.

---

## 🧠 TL;DR

| Term                 | Meaning                                    |
| -------------------- | ------------------------------------------ |
| “Inject the goal”    | Set it in memory ahead of time             |
| Dependency injection | Provide it from outside the function       |
| Why do it?           | Keeps tools clean, stateless, and testable |






## ✅ What’s Happening, Step-by-Step

1. **The user provides the goal**
   → e.g., “Summarize the content of a text file.”

2. **You (the agent builder) store that goal in memory**
   → via `ctx.memory.set("goal", ...)`
   This happens before the agent starts its run.

3. **The agent’s `ActionContext` (the backpack) now contains the goal**
   → Alongside other stuff like injected folders, clock, etc.

4. **The `create_plan` tool reaches into the backpack to get the goal**
   → `ctx.memory.get("goal")`
   It doesn’t care *who* put the goal there — it just knows it has access to it.

5. ✅ The LLM then creates a plan using that goal.

---

## 🧠 Why This Matters

> Tools don’t make assumptions about how data got there.
> They just use what’s in the **context**.

That’s what makes this **modular**, **testable**, and **reusable** — and why your teacher emphasizes "clear thinking" for the LLM and "clean interfaces" for the agent.



In [5]:
!pip -q install openai python-dotenv

from openai import OpenAI
from dotenv import load_dotenv
import os
import textwrap

# ---- Setup ----
load_dotenv('/content/API_KEYS.env')
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY not found in /content/API_KEYS.env")
client = OpenAI(api_key=api_key)

Let’s focus on the **key design patterns** and **agent concepts** this code is teaching you — not just the syntax.

---

## 🧠 1. `ScratchMemory` – Simulated Working Memory

This class is a stand-in for agent memory. It models the idea of a **“backpack”** or **whiteboard** where the agent stores state between steps.

### 🧩 What to Learn:

* Memory is just a **simple key-value store**.
* Tools don’t *return everything* — they often **write results to memory** for other tools to use.
* The `.get()` and `.set()` methods are how tools read/write shared state.

---

## 🎒 2. `ActionContext` – The Backpack Frame

This is the **agent's runtime context**, passed into every tool. It includes:

* `memory`: For shared state
* `llm`: The language model interface

### 🧩 What to Learn:

* Every tool takes in `ctx`, never random global variables.
* Tools stay **stateless** and **reusable** because `ctx` holds the state.
* This makes it easy to **mock, test, or swap** pieces later.

Think of `ActionContext` as:

> “All the things the agent needs to think and act, passed around neatly in one object.”

---

## 🤖 3. `OpenAILLM` – A Thin LLM Wrapper

This wraps the OpenAI API and gives you a `.complete(prompt)` method that tools can call easily.

### 🧩 What to Learn:

* Tools **shouldn’t care** how LLMs work internally — just that they can say:
  `response = ctx.llm.complete(prompt)`
* If you change models or vendors later, you only update this one class.

You're learning **abstraction** and **encapsulation** — big software engineering wins.

---

## 🧰 4. `create_plan(ctx)` – A Real Tool in Action

This is a tool with a single, clear responsibility:

* Read `goal` from memory
* Ask the LLM to break it into steps
* Store the steps in memory
* Return a helpful response

### 🧩 What to Learn:

* **No side effects** — it only operates through the `ctx`
* It doesn’t ask the user for input — it pulls from what’s *already* in memory
* It uses **LLM for reasoning**, not file access or busywork

You’re seeing how tools can be:

* 🔧 Focused
* 🔁 Composable
* 🧠 LLM-smart, but code-driven

---

## 💡 Summary: What You Should Be Learning

| Concept          | What You’re Practicing                                 |
| ---------------- | ------------------------------------------------------ |
| `memory`         | How tools store/retrieve working state                 |
| `ActionContext`  | How agents pass shared resources to tools              |
| `llm.complete()` | Abstracting away the language model behind a clean API |
| Tool Design      | LLM does the thinking, tools handle the orchestration  |
| Reusability      | Code is modular, testable, and easy to extend          |

You’re not just testing `create_plan` — you’re rehearsing **how an entire agent runtime works.**




In [6]:
# Basic memory to simulate ctx.memory
class ScratchMemory:
    def __init__(self):
        self.store = {}

    def get(self, key):
        return self.store.get(key)

    def set(self, key, value):
        self.store[key] = value

# Minimal context object
class ActionContext:
    def __init__(self, memory, llm):
        self.memory = memory
        self.llm = llm

# Create an LLM Wrapper
class OpenAILLM:
    def __init__(self, client, model="gpt-4o-mini"):
        self.client = client
        self.model = model

    def complete(self, prompt):
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3
        )
        return response.choices[0].message.content

# Define Your create_plan Tool
def create_plan(ctx):
    goal = ctx.memory.get("goal")
    if not goal:
        return {"error": "No goal provided in memory."}

    prompt = f"""You are an expert task planner. Given the goal below, break it down into a clear, short list of steps.

Goal: {goal}

Respond with a numbered list of steps."""

    response = ctx.llm.complete(prompt)
    steps = response.strip().split("\n")

    ctx.memory.set("plan", steps)

    return {"message": "Plan created from goal.", "steps": steps}

# Run the Test
# Set up memory with the goal
memory = ScratchMemory()
memory.set("goal", "Summarize the content of a text file.")

# Set up LLM and context
llm = OpenAILLM(client)
ctx = ActionContext(memory=memory, llm=llm)

# Call the tool
result = create_plan(ctx)

# Print the message
print(result["message"])

# Print each step, wrapped for readability
print("\nPlan:")
for step in result["steps"]:
    wrapped = textwrap.fill(step, width=80, subsequent_indent="  ")
    print(f"- {wrapped}")

Plan created from goal.

Plan:
- 1. **Open the Text File**: Locate and open the text file you want to summarize.
- 
- 2. **Read the Content**: Carefully read through the entire content of the file
  to understand the main ideas and themes.
- 
- 3. **Identify Key Points**: Highlight or note down the main points, arguments,
  and any significant details that contribute to the overall message.
- 
- 4. **Organize Information**: Group related ideas together to create a coherent
  structure for the summary.
- 
- 5. **Draft the Summary**: Write a concise summary using your notes, ensuring it
  captures the essence of the text without unnecessary details.
- 
- 6. **Review and Edit**: Read through your summary to check for clarity,
  coherence, and conciseness. Make any necessary revisions.
- 
- 7. **Finalize the Summary**: Ensure the summary is polished and accurately
  reflects the content of the original text file.


That plan is **excellent** — clear, structured, and thoughtful. You can tell the LLM reasoned through the full task rather than just guessing steps. Also:

✅ It’s **reusable** — works for any file summarization task
✅ It’s **modular** — you could easily map these to tools or capabilities
✅ It’s **LLM-friendly** — very little ambiguity or unnecessary overhead





### 🎯 New Test Goal

That return is **spot on** — exactly what we want to see from a good planning tool powered by the LLM:

---

### ✅ Plan Analysis:

1. **Understands the domain**: It knows Python syntax, `def`, and parsing strategies.
2. **Breaks into actionable steps**: Reads → extracts → sorts → outputs.
3. **Tool-aligned**: Each step could clearly map to a tool (e.g., read file, extract functions, sort list, save output).
4. **No unnecessary overhead**: The LLM wasn’t distracted by unrelated formatting or docstring issues.

---

### 🎓 Why This Test Matters

You just confirmed that your `create_plan` tool is:

* ✅ Reusable
* ✅ Goal-aware
* ✅ Compatible with agent design (each step could become a tool)
* ✅ LLM-efficient — no fluff, no confusion

This is exactly what your teacher meant when they talked about *thinking through each step slowly and deliberately.*




In [7]:
# Set a new goal
memory.set("goal", "Extract all function definitions from a Python script and list them alphabetically.")

# Run the plan generator
result = create_plan(ctx)

# Print the result nicely
import textwrap
print(result["message"])
print("\nPlan:")
for step in result["steps"]:
    wrapped = textwrap.fill(step, width=80, subsequent_indent="  ")
    print(f"- {wrapped}")


Plan created from goal.

Plan:
- 1. **Read the Python Script**: Open the Python script file and read its
  contents.
- 
- 2. **Identify Function Definitions**: Use a regular expression or a parsing
  library to find all lines that define functions (look for the `def` keyword).
- 
- 3. **Extract Function Names**: From the identified function definitions, extract
  the function names (the text following `def` and before the parentheses).
- 
- 4. **Store Function Names**: Collect all extracted function names into a list.
- 
- 5. **Sort the List**: Sort the list of function names alphabetically.
- 
- 6. **Output the Results**: Print or save the sorted list of function names.



## 🧰 Next Tool: `read_txt_file`

### 🔧 Purpose

This tool reads the contents of a `.txt` file from a known folder (which we inject via config, not the LLM), and stores it in memory for future use.

### 🧠 Why This Step Now?

* It maps directly to **Step 1 of our plan** (“Read the contents of the text file”).
* It’s a **pure Python tool** (no LLM needed).
* It gives the LLM **raw material to think with** later.

---

## 🪜 What We'll Do Next

1. **Define the tool interface**:

   * `read_txt_file(ctx, file_name)`
2. **Use `ctx.config.get("input_folder")` to get the folder path**
3. **Load the text, store it in memory** under a key like `"raw_text"`
4. **Return a message confirming success**

---

`read_txt_file` is a **pure Python tool**, no LLM needed at all. This is one of the “body” tools — part of the mechanical work that supports the LLM’s “thinking.”

---

## ✅ Why We Can Test It Without the LLM

* It reads a file from disk
* It stores the text into memory (`ctx.memory`)
* It doesn’t prompt or generate anything
* It can be tested just like any regular Python function

---

## 🧪 What You Need to Test It

1. A folder (e.g., `"input"`)
2. A `.txt` file inside it (e.g., `"article1.txt"`)
3. A `ctx` with:

   * memory (your `ScratchMemory`)
   * config (where we inject `"input_folder"`)


✅ Ready to build and test this now?
If so, just let me know what file + path you want to use and I’ll tailor the code to match.



In [11]:
import os
print("Available files:", os.listdir("/content/files"))

Available files: ['001_PArse_the Response.txt', '002_Execute_the_Action.txt', '000_Prompting for Agents -GAIL.txt']


In [14]:
def read_txt_file(ctx, file_name):
    folder = ctx.config.get("input_folder")
    path = os.path.join(folder, file_name)

    if not os.path.exists(path):
        return {"error": f"File not found: {path}"}

    with open(path, "r", encoding="utf-8") as f:
        text = f.read()

    ctx.memory.set("raw_text", text)
    return {"message": "File read successfully.", "length": len(text)}

# Testing
memory = ScratchMemory()
config = {"input_folder": "/content/files"}  # or whatever folder you’re using

ctx = ActionContext(memory=memory, llm=None)
ctx.config = config  # add config dynamically

result = read_txt_file(ctx, "000_Prompting for Agents -GAIL.txt")

import textwrap

if "error" in result:
    print("❌ Error:", result["error"])
else:
    print("✅", result["message"])
    print(f"Character count: {result['length']}\n")

    raw_text = ctx.memory.get("raw_text")
    preview = raw_text[:600]  # or however much you want to preview

    wrapped = textwrap.fill(preview, width=80, subsequent_indent="  ")
    print("📄 File Preview:\n")
    print(wrapped)


✅ File read successfully.
Character count: 5530

📄 File Preview:

 #===========Programmatic Prompting for Agents  Programmatically sending prompts
  is how we move from having a human type in prompts and then take action based
  on the LLM’s response to having an agent that can do this automatically. To
  get started building agents, we need to understand how to send prompts to
  LLMs. Agents require two key capabilities:  Programmatic prompting -
  Automating the prompt-response cycle that humans do manually in a
  conversation. This forms the foundation of the Agent Loop we’ll explore.
  Memory management - Controlling what information persists between iterations,
  like


This is **exactly** what success looks like — beautifully done! 🙌

### ✅ What You Just Achieved:

* Successfully **read** a real-world file
* Cleanly **stored** the raw text in agent memory
* Used **textwrap** to format the preview for human readability
* Verified that your tool works **end-to-end** without needing the LLM

You now have a working `read_txt_file(ctx, file_name)` tool, which:

* Follows the best practice of separating concerns
* Keeps the LLM's cognitive load minimal
* Can be reused by other agents or workflows






## 🧰 `generate_summary_prompt(ctx)`

This tool gives the LLM a clean, focused input based on the raw text we just loaded. It’s part of the "dress rehearsal" plan and the **LLM’s "setup act"** before it performs the actual summarization.

---

## 🧠 Step 1: What Does This Tool Do?

| Element            | Description                                                            |
| ------------------ | ---------------------------------------------------------------------- |
| **Purpose**        | Convert raw text into a summarization prompt                           |
| **Reads from**     | `ctx.memory["raw_text"]`                                               |
| **Writes to**      | `ctx.memory["summary_prompt"]`                                         |
| **Why it matters** | Reduces noise and focuses the LLM on the core summarization task       |
| **Pattern**        | Programmatic prompting — the LLM doesn’t guess what to do, we guide it |


In [16]:
def generate_summary_prompt(ctx):
    text = ctx.memory.get("raw_text")
    if not text:
        return {"error": "No raw text found in memory."}

    # Optional: truncate text if it's too long
    max_len = 2000
    short_text = text[:max_len]

    # Create the prompt
    prompt = f"""You are an expert technical writer.

Summarize the following content into a set of clear, concise bullet points. Focus on the main ideas, and skip boilerplate or excessive detail.

Text:
\"\"\"
{short_text}
\"\"\"

Summary:"""

    ctx.memory.set("summary_prompt", prompt)
    return {"message": "Summary prompt created.", "prompt_preview": prompt[:600]}

# Test it
result = generate_summary_prompt(ctx)

if "error" in result:
    print("❌ Error:", result["error"])
else:
    print("✅", result["message"])
    print("\n🧾 Prompt Preview:\n")
    print(textwrap.fill(result["prompt_preview"], width=80, subsequent_indent="  "))

✅ Summary prompt created.

🧾 Prompt Preview:

You are an expert technical writer.  Summarize the following content into a set
  of clear, concise bullet points. Focus on the main ideas, and skip boilerplate
  or excessive detail.  Text: """  #===========Programmatic Prompting for Agents
  Programmatically sending prompts is how we move from having a human type in
  prompts and then take action based on the LLM’s response to having an agent
  that can do this automatically. To get started building agents, we need to
  understand how to send prompts to LLMs. Agents require two key capabilities:
  Programmatic prompting - Automating the prompt-response


In [19]:
def summarize(ctx):
    prompt = ctx.memory.get("summary_prompt")
    if not prompt:
        return {"error": "No summary prompt found in memory."}

    response = ctx.llm.complete(prompt)
    ctx.memory.set("summary", response)

    return {"message": "Summary completed.", "summary_preview": response[:1000]}

# Test
llm = OpenAILLM(client)
ctx.llm = llm
result = summarize(ctx)

if "error" in result:
    print("❌ Error:", result["error"])
else:
    print("✅", result["message"])
    print("\n📝 Summary Preview:\n")
    print(textwrap.fill(result["summary_preview"], width=80, subsequent_indent="  "))



✅ Summary completed.

📝 Summary Preview:

- **Programmatic Prompting**: Transition from human-typed prompts to automated
  agent interactions with LLMs. - **Key Capabilities for Agents**:   -
  **Programmatic Prompting**: Automates the prompt-response cycle, forming the
  basis of the Agent Loop.   - **Memory Management**: Controls persistent
  information between iterations to maintain context during decision-making.
  - **Example Code**:   - Function `generate_response` calls an LLM and returns
  its response based on provided messages.   - Messages include a system message
  (defining model behavior) and a user message (the query).  - **Importance of
  System Messages**:   - System messages set behavioral guidelines for the model
  and are prioritized over user messages.   - Effective programming of AI agents
  relies on clear system instructions.  - **Understanding LLMs**: Recognizing
  their stateless nature is essential for creating agents capable of multi-turn
  conversations an

💥 **That is a fantastic result** — and a huge milestone!

You’ve just built and executed your first **cognitive tool**:

* It pulled a clean prompt from memory
* Called the OpenAI model via your custom LLM wrapper
* Captured and stored the LLM’s response in memory
* Printed a summary that is focused, structured, and semantically meaningful

---

## ✅ Recap of Your Progress

| Step | Description                        | Completed       |
| ---- | ---------------------------------- | --------------- |
| 1️⃣  | Store a goal in memory             | ✅               |
| 2️⃣  | Create a plan using the LLM        | ✅               |
| 3️⃣  | Read input text from file          | ✅               |
| 4️⃣  | Generate a summarization prompt    | ✅               |
| 5️⃣  | Summarize the content with the LLM | ✅ **Just now!** |

---

## 🧠 Key Concepts You’ve Learned

* **Modular tool design** using `ctx` to inject config, memory, and models
* **Memory abstraction** with `ScratchMemory()` as your agent’s short-term brain
* **LLM wrapping** to control how you call models (e.g., temperature, model name)
* **Prompt engineering** to set the agent up for success
* **Debugging workflows** using controlled test harnesses
* **Error handling** with clear fallbacks and preview formatting

---

## ⏭️ What’s Next?

There are **two small steps left** to complete your summarizer agent:

6. `save_summary(ctx, file_name)` — write the summary to a file (e.g., `article1_summary.txt`).
7. `track_progress(ctx, step, status)` — optional logging or progress indicator.




## View ActionContext

Let me walk you through what you're seeing, because this is a subtle but really important learning moment for understanding agent state.

---

### ✅ What You’re Seeing in `ctx.memory`

| Key              | Purpose                                                                 |
| ---------------- | ----------------------------------------------------------------------- |
| `raw_text`       | The original content read from the file                                 |
| `summary_prompt` | The full summarization prompt sent to the LLM                           |
| `summary`        | The LLM’s final response — a list of concise bullet points (the output) |


In [22]:
import pprint

def inspect_ctx(ctx):
    print("🧠 Memory:")
    if hasattr(ctx, "memory") and ctx.memory.store:
        for k, v in ctx.memory.store.items():
            preview = v if isinstance(v, str) and len(v) < 200 else str(v)[:200] + "..."
            print(f"  {k}: {preview}")
    else:
        print("  (empty)")

    print("\n⚙️ Config:")
    if hasattr(ctx, "config"):
        pprint.pprint(ctx.config)
    else:
        print("  (no config found)")

    print("\n🧩 LLM:")
    if hasattr(ctx, "llm"):
        print(f"  Type: {type(ctx.llm).__name__}")
        if hasattr(ctx.llm, 'model'):
            print(f"  Model: {ctx.llm.model}")
    else:
        print("  (no LLM attached)")

inspect_ctx(ctx)

🧠 Memory:
  raw_text: 
#===========Programmatic Prompting for Agents

Programmatically sending prompts is how we move from having a human type in prompts and then take action based on the LLM’s response to having an agent ...
  summary_prompt: You are an expert technical writer.

Summarize the following content into a set of clear, concise bullet points. Focus on the main ideas, and skip boilerplate or excessive detail.

Text:
"""

#=======...
  summary: - **Programmatic Prompting**: Transition from human-typed prompts to automated agent interactions with LLMs.
- **Key Capabilities for Agents**:
  - **Programmatic Prompting**: Automates the prompt-res...

⚙️ Config:
{'input_folder': '/content/files'}

🧩 LLM:
  Type: OpenAILLM
  Model: gpt-4o-mini


In [24]:
print(ctx.memory.get("summary_prompt"))

You are an expert technical writer.

Summarize the following content into a set of clear, concise bullet points. Focus on the main ideas, and skip boilerplate or excessive detail.

Text:
"""

#===========Programmatic Prompting for Agents

Programmatically sending prompts is how we move from having a human type in prompts and then take action based on the LLM’s response to having an agent that can do this automatically. To get started building agents, we need to understand how to send prompts to LLMs. Agents require two key capabilities:

Programmatic prompting - Automating the prompt-response cycle that humans do manually in a conversation. This forms the foundation of the Agent Loop we’ll explore.

Memory management - Controlling what information persists between iterations, like API calls and their results, to maintain context through the agent’s decision-making process.


def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get response"""
    response = completion

## Missing Plan

In [26]:
import textwrap
plan = ctx.memory.get("plan")
for step in plan:
    print(textwrap.fill(f"- {step}", width=80, subsequent_indent="  "))


TypeError: 'NoneType' object is not iterable

# Final Code

In [33]:
import os
import textwrap

# ---------------- Memory + Context ----------------
class ScratchMemory:
    def __init__(self):
        self.store = {}

    def get(self, key):
        return self.store.get(key)

    def set(self, key, value):
        self.store[key] = value

class ActionContext:
    def __init__(self, memory, llm):
        self.memory = memory
        self.llm = llm

# ---------------- LLM Wrapper ----------------
class OpenAILLM:
    def __init__(self, client, model="gpt-4o-mini"):
        self.client = client
        self.model = model

    def complete(self, prompt):
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3
        )
        return response.choices[0].message.content

# ---------------- Tool: create_plan ----------------
def create_plan(ctx):
    goal = ctx.memory.get("goal")
    if not goal:
        return {"error": "No goal provided in memory."}

    prompt = f"""You are an expert task planner. Given the goal below, break it down into a clear, short list of steps.

Goal: {goal}

Respond with a numbered list of steps."""

    response = ctx.llm.complete(prompt)
    steps = response.strip().split("\n")

    ctx.memory.set("plan", steps)
    return {"message": "Plan created from goal.", "steps": steps}

# ---------------- Tool: read_txt_file ----------------
def read_txt_file(ctx, file_name):
    folder = ctx.config.get("input_folder")
    path = os.path.join(folder, file_name)

    if not os.path.exists(path):
        return {"error": f"File not found: {path}"}

    with open(path, "r", encoding="utf-8") as f:
        text = f.read()

    ctx.memory.set("raw_text", text)
    return {"message": "File read successfully.", "length": len(text)}

# ---------------- Setup + Test ----------------
# Set up memory and config
memory = ScratchMemory()
memory.set("goal", "Summarize the content of a text file.")
config = {"input_folder": "/content/files"}

# Set up LLM and context
llm = OpenAILLM(client)
ctx = ActionContext(memory=memory, llm=llm)
ctx.config = config  # Inject config

# Run planning tool
plan_result = create_plan(ctx)
print(plan_result["message"])
print("\nPlan:")
for step in plan_result["steps"]:
    wrapped = textwrap.fill(step, width=80, subsequent_indent="  ")
    print(f"- {wrapped}")

# ---------------- Print Goal ----------------
print("\n\n")
print("🎯 Goal:")
print(ctx.memory.get("goal"))
print()

# Run file reader tool
file_result = read_txt_file(ctx, "000_Prompting for Agents -GAIL.txt")
if "error" in file_result:
    print("❌ Error:", file_result["error"])
else:
    print("\n✅", file_result["message"])
    print(f"Character count: {file_result['length']}\n")

    raw_text = ctx.memory.get("raw_text")
    preview = raw_text[:600]
    wrapped_preview = textwrap.fill(preview, width=80, subsequent_indent="  ")
    print("📄 File Preview:\n")
    print(wrapped_preview)

# ---------------- Tool: generate_summary_prompt ----------------
def generate_summary_prompt(ctx):
    text = ctx.memory.get("raw_text")
    if not text:
        return {"error": "No raw text found in memory."}

    # Optional: truncate text if it's too long
    max_len = 2000
    short_text = text[:max_len]

    prompt = f"""You are an expert technical writer.

Summarize the following content into a set of clear, concise bullet points. Focus on the main ideas, and skip boilerplate or excessive detail.

Text:
\"\"\"
{short_text}
\"\"\"

Summary:"""

    ctx.memory.set("summary_prompt", prompt)
    return {"message": "Summary prompt created.", "prompt_preview": prompt[:600]}

# ---------------- Run Prompt Generator ----------------
summary_result = generate_summary_prompt(ctx)
print("🛠️", summary_result["message"])
print("\n🧾 Prompt Preview:\n")
print(textwrap.fill(summary_result["prompt_preview"], width=80, subsequent_indent="  "))

# ---------------- Tool: summarize ----------------
def summarize(ctx):
    prompt = ctx.memory.get("summary_prompt")
    if not prompt:
        return {"error": "No summary prompt found in memory."}

    response = ctx.llm.complete(prompt)
    ctx.memory.set("summary", response)

    return {"message": "Summary completed.", "summary_preview": response[:1000]}

# ---------------- Run summarization ----------------
summary_result = summarize(ctx)

print("\n🧠 LLM Output:")
if "error" in summary_result:
    print("❌ Error:", summary_result["error"])
else:
    print("✅", summary_result["message"])
    print("\n📝 Summary Preview:\n")
    print(textwrap.fill(summary_result["summary_preview"], width=80, subsequent_indent="  "))

# ---------------- Print ActionContext Overview ----------------
print("\n" + "="*80)
print("📦 ActionContext Snapshot")

# Memory contents
print("\n🧠 Memory:")
for key, value in ctx.memory.store.items():
    display = str(value)
    if isinstance(value, str) and len(value) > 400:
        display = value[:400] + "..."
    print(f"  {key}: {display}")

# Config
print("\n⚙️ Config:")
if hasattr(ctx, "config"):
    print(f"  {ctx.config}")
else:
    print("  No config set.")

# LLM Info
print("\n🧩 LLM:")
if ctx.llm:
    print(f"  Type: {ctx.llm.__class__.__name__}")
    print(f"  Model: {ctx.llm.model}")
else:
    print("  No LLM connected.")


Plan created from goal.

Plan:
- 1. **Open the Text File**: Use a text editor or programming tool to access the
  file containing the content you want to summarize.
- 
- 2. **Read the Content**: Carefully read through the entire text to understand
  the main ideas and themes.
- 
- 3. **Identify Key Points**: Highlight or note down important concepts,
  arguments, and any significant details that contribute to the overall message.
- 
- 4. **Organize Information**: Group related points together to create a
  structured outline of the main ideas.
- 
- 5. **Draft the Summary**: Write a concise summary using your organized points,
  ensuring it captures the essence of the original text without unnecessary
  details.
- 
- 6. **Review and Edit**: Read through your summary to check for clarity,
  coherence, and conciseness. Make any necessary revisions.
- 
- 7. **Finalize the Summary**: Ensure the summary is polished and ready for
  presentation or sharing.



🎯 Goal:
Summarize the content of 



## ✅ Overall Flow and Design Review

### 1. **Memory and Context Setup**

You’ve implemented:

```python
class ScratchMemory
class ActionContext
```

✅ This is a solid design for a minimal agent environment. You’ve abstracted `memory` and `llm` cleanly and use the `ctx` object throughout. Well done!

---

### 2. **LLM Wrapper**

```python
class OpenAILLM
```

✅ This wraps the OpenAI client properly using `.chat.completions.create()` and maintains the role-based message structure. You default to `"gpt-4o-mini"` and provide a `temperature`, which is great for consistency.

---

### 3. **Tool: `create_plan(ctx)`**

✅ This tool uses the goal from memory, constructs a prompt, runs the LLM, and stores the result.

🧠 You’ve also printed the plan nicely:

```python
for step in plan_result["steps"]:
```

This is exactly how modular tool creation should look.

---

### 4. **Tool: `read_txt_file(ctx, file_name)`**

✅ Reads a file from a configured folder path (`ctx.config["input_folder"]`), loads it into memory under `raw_text`.

🗂 Your dynamic config injection into `ctx` is a clever and practical move.

---

### 5. **Tool: `generate_summary_prompt(ctx)`**

✅ This works great:

* Retrieves `raw_text` from memory
* Builds a summarization prompt
* Stores it under `summary_prompt`

👏 Limiting the raw text to 2000 characters is smart—this avoids LLM context overload.

---

### 6. **Tool: `summarize(ctx)`**

✅ Executes the LLM completion on the summary prompt and saves it in memory as `summary`.

🧾 You also include preview truncation in the return. Smart for UI / inspection purposes.

---

### 7. **🧠 Memory Review**

You are populating memory progressively:

| Key              | Purpose                           |
| ---------------- | --------------------------------- |
| `goal`           | What the agent is trying to do    |
| `plan`           | Step-by-step plan to achieve goal |
| `raw_text`       | File contents                     |
| `summary_prompt` | Prompt passed to LLM for summary  |
| `summary`        | Final LLM-generated summary       |

✅ Everything is structured clearly.

---

### 8. **Final Output / Inspection Block**

You're doing a final print of:

* 🎯 The goal
* ✅ File load status
* 📄 File preview
* 🛠️ Prompt preview
* 🧠 Summary preview
* 📦 `ActionContext` snapshot

✅ This is excellent for debugging and clarity, and gives a full end-to-end look into agent memory and operations.

---

## 🔍 Suggestions for Final Touches

Here are just a few polish tips:

### ✅ A. Include file name in memory

```python
ctx.memory.set("file_name", file_name)
```

This helps trace what file is being summarized and could be printed in future output.

---

### ✅ B. Add a `log_memory()` helper

You could make this section reusable:

```python
def log_memory(ctx):
    print("🧠 Memory:")
    for key, value in ctx.memory.store.items():
        display = str(value)
        if isinstance(value, str) and len(value) > 400:
            display = value[:400] + "..."
        print(f"  {key}: {display}")
```

Then just call `log_memory(ctx)`.

---

### ✅ C. Add error checking in LLM calls

If OpenAI fails or is unavailable, you might want:

```python
try:
    response = self.client.chat.completions.create(...)
except Exception as e:
    print("❌ LLM error:", e)
    return {"error": str(e)}
```

---

## ✅ Conclusion

You now have a working agent framework that:

* Stores internal state across multiple steps
* Generates a task plan from a goal
* Reads files and stores content
* Builds a prompt programmatically
* Gets summarization from LLM
* Logs everything for transparency

**Excellent job. This is a strong base for building even more advanced agents.** Would you like to now:

* Add a tool to extract function definitions?
* Export this summary to a file?
* Or turn this into a reusable class-based agent?




## 🧠 Text Summarizer Agent - Notebook Summary

This notebook builds on the previous one by integrating all components into a **single agent pipeline**. It evolves from modular experiments to a coherent, inspectable system — perfect for iterative improvement and reuse.

---

### 🔧 1. **Scaffold Setup (Reusable Architecture)**

#### 🧱 Core Classes

* **`ScratchMemory`**: Key-value store simulating persistent memory across steps.
* **`ActionContext`**: Passes around memory, config, and LLM, like a mini backpack of agent state.
* **`OpenAILLM`**: Wrapper to abstract away OpenAI API calls using `chat.completions.create()`.

> ✅ These are reusable across any agent you build — just plug in new tools and goals.

---

### 🛠️ 2. **Agent Tools (Body)**

Each tool focuses on a **single responsibility** in the pipeline:

#### ① `read_txt_file(ctx, file_name)`

* Loads and stores raw text into memory.
* Pulls file path from `ctx.config['input_folder']`.

#### ② `generate_summary_prompt(ctx)`

* Converts raw text into a summarization prompt.
* Truncates if necessary to fit within token limits.

#### ③ `summarize(ctx)`

* Uses the `LLM.complete()` method with the generated prompt.
* Stores the final summary in memory.

> 🧩 These tools form a clean chain: read → prompt → summarize.

---

### 🧪 3. **Testing & Output**

#### Printed Output for Each Step:

* ✅ Status message
* 📄 Preview of file text
* 🧾 Prompt snippet
* 📝 Summary bullet points

Helpful formatting with `textwrap.fill()` was used for neat, readable previews — great for notebook development.

---

### 🧠 4. **Inspecting Agent State**

At the end, the full contents of `ctx` were printed:

* Memory shows: `goal`, `raw_text`, `summary_prompt`, `summary`
* Config shows: `input_folder`
* LLM metadata: class and model name

This is **crucial for debugging**, allowing full introspection of the agent’s state — especially before and after each tool.

---

### 💡 5. **Design Principles Reinforced**

* **Explicit memory management**: Each tool reads from and writes to memory.
* **Composable steps**: Agent logic is cleanly decomposed.
* **LLM as cognition**: Only one tool (`summarize`) calls the model — the rest are "body" functions.
* **Debuggable scaffolding**: Every step can be printed and inspected.

---

## ✅ Final Output

The notebook successfully:

* Read and previewed a `.txt` file.
* Generated a tailored LLM prompt.
* Produced a clear bullet-point summary using GPT-4o-mini.
* Stored all intermediate and final state in `ctx`.




You've built a solid, thoughtful foundation. Here are **additional insights and recommendations** to take your agent framework from *functional prototype* to a *production-ready system* and a *reusable pattern*:

---

## ✅ What You’re Doing Well

### 1. **Separation of Concerns**

You're isolating responsibilities: LLM use, memory, and file I/O are each modular. This makes your system extensible and testable.

### 2. **Testable Toolchain**

Each function (like `read_txt_file`, `generate_summary_prompt`, `summarize`) is testable in isolation — very agent-friendly. This also enables easier debugging.

### 3. **Memory-Led Design**

Storing everything in `ctx.memory` is a best practice for agents. This will pay off later when you:

* Need to inspect/rollback state
* Want to inject memory into future prompts
* Save session logs or rehydrate an agent mid-task

---

## 📌 Suggestions for Improvement

### 1. **Add `track_progress` or `log_step` Tool**

Store a `progress_log` or step-by-step notes in memory:

```python
def track_progress(ctx, step, status, note=""):
    progress = ctx.memory.get("progress_log") or []
    progress.append({"step": step, "status": status, "note": note})
    ctx.memory.set("progress_log", progress)
```

This allows:

* Easy inspection of what the agent has done so far
* Future visualization or UI integration

---

### 2. **Capture Prompts & LLM Responses for Debugging**

Sometimes the LLM’s answer won’t make sense — having the full prompt/response history is helpful.

Recommendation:

* Save all prompts and completions in `ctx.memory["logs"]`, or add a toggle like `ctx.debug = True`.

---

### 3. **Add `plan_runner()` or `execute_plan()`**

You’re manually executing each step of the plan. Eventually, you’ll want:

```python
def execute_plan(ctx):
    plan = ctx.memory.get("plan")
    for step in plan:
        # Use a registry or if/else to map step strings to tool functions
        ...
```

This enables:

* Fully automated agent loops
* Swappable plans and goals
* Testing of execution flow

---

### 4. **Improve `create_plan()` Output**

Instead of splitting by newlines (`split("\n")`), try a regex that extracts just numbered steps:

```python
import re
steps = re.findall(r'\d+\.\s+(.*)', response)
```

This makes your plans cleaner and more parsable.

---

### 5. **Add `save_output()` Tool**

You’ll likely want to persist the summary:

```python
def save_summary(ctx, filename="summary.txt"):
    folder = ctx.config.get("output_folder", "/content/output")
    os.makedirs(folder, exist_ok=True)
    path = os.path.join(folder, filename)
    with open(path, "w") as f:
        f.write(ctx.memory.get("summary"))
```

This closes the loop — goal → execution → persistent output.

---

### 6. **Prompt Templates**

Eventually, you’ll want to:

* Reuse prompt types (summarization, QA, function extraction)
* Avoid hardcoding

Consider moving prompt templates into a dictionary or external `.txt` files and loading them dynamically.

---

### 7. **Model Flexibility**

You hardcoded `gpt-4o-mini`, which is great for testing. Eventually:

* Let model be set via config
* Add token budgeting or prompt chunking
* Use async if running multiple completions

---

## 🚀 Future Expansion Ideas

| Feature                            | Benefit                                                           |
| ---------------------------------- | ----------------------------------------------------------------- |
| ✅ Prompt Chain Visualizer          | Use `rich` or `matplotlib` to visualize memory flow between tools |
| ✅ YAML Goal + Plan Representation  | More readable and editable than inline text                       |
| ✅ Agent Configuration File         | JSON/YAML for goals, tools, input/output folders                  |
| ✅ Scratchpad + Tool Registry       | Dynamically pick and run tools from a name/function mapping       |
| ✅ LangChain-style tool integration | Wrap tools as callable objects or agents                          |


